In [16]:
import matplotlib.pyplot as plt
import seaborn as sns
import re
import subprocess
import numpy as np
import json

In [17]:
!bash -c 'cd /home/Zsf/HPC/hw1 && source /opt/intel/oneapi/setvars.sh > /dev/null && make parallel NTHREADS=20 OMP_SCHEDULE=dynamic SIZE=20000 && ./release/omp_homework'

icx -g -Wall -qopenmp -qopt-report=3 -march=native -O3 -ffast-math \
	-D SIZE=20000 \
	-D PARALLEL \
	-D OMP_SCHEDULE=dynamic \
	-D NTHREADS=20 \
	./src/omp_homework.c -o ./release/omp_homework 
DFTW calculation with N = 20000 
DFTW computation in 0.180172 seconds
Xre[0] = 20000.000000 


In [18]:
re_pattern = r"DFTW computation in (\d+\.\d+) seconds"
SIZES = [20000, 30000, 40000]
RUNS = 1

# CPU best params

In [19]:
NUM_THREADS_LIST = [1, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
# NUM_THREADS_LIST = [1, 2, 4, 6]
SCHEDULE_LIST = ["static", "dynamic", "guided"]

In [21]:
import subprocess
import re
import numpy as np
import json

best_cpu_param_times = np.zeros((len(NUM_THREADS_LIST), len(SCHEDULE_LIST), len(SIZES)))

results_dict = {}

for run in range(RUNS):
    print(f"Run {run + 1}")
    for r, size in enumerate(SIZES):
      for i, nthreads in enumerate(NUM_THREADS_LIST):
          for j, schedule in enumerate(SCHEDULE_LIST):
              print(f"nthreads={nthreads}, schedule={schedule}, size={size}")
              
              # Modify the command to run in a single bash shell
              command = f"bash -c 'cd /home/Zsf/HPC/hw1 && source /opt/intel/oneapi/setvars.sh > /dev/null && make parallel NTHREADS={nthreads} OMP_SCHEDULE={schedule} SIZE={size} && ./release/omp_homework'"
      
              # Run the command and capture both stdout and stderr
              result = subprocess.run(command, shell=True, capture_output=True, text=True)
              
              # Print both stdout and stderr
              print("STDOUT:")
              print(result.stdout)
              print("STDERR:")
              print(result.stderr)
              
              # Check if the elapsed time matches the regex
              match = re.search(re_pattern, result.stdout)
              if match:
                  time_elapsed = float(match.group(1))
                  best_cpu_param_times[i, j, r] = time_elapsed
                  
                  # Add the result to the dictionary
                  if size not in results_dict:
                        results_dict[size] = {}
                  if nthreads not in results_dict[size]:
                      results_dict[size][nthreads] = {}
                  results_dict[size][nthreads][schedule] = time_elapsed
              else:
                  print(f"[DEBUG] No match found in stdout for elapsed time.")

# Average the results over the number of runs
best_cpu_param_times /= RUNS

# Save the results dictionary to a JSON file
output_filename = "./data/cpu_param_times.json"
with open(output_filename, 'w') as f:
    json.dump(results_dict, f, indent=4)

print(f"Results saved to {output_filename}")


Run 1
nthreads=1, schedule=static, size=20000
STDOUT:
icx -g -Wall -qopenmp -qopt-report=3 -march=native -O3 -ffast-math \
	-D SIZE=20000 \
	-D PARALLEL \
	-D OMP_SCHEDULE=static \
	-D NTHREADS=1 \
	./src/omp_homework.c -o ./release/omp_homework 
DFTW calculation with N = 20000 
DFTW computation in 1.726435 seconds
Xre[0] = 20000.000000 

STDERR:

nthreads=1, schedule=dynamic, size=20000
STDOUT:
icx -g -Wall -qopenmp -qopt-report=3 -march=native -O3 -ffast-math \
	-D SIZE=20000 \
	-D PARALLEL \
	-D OMP_SCHEDULE=dynamic \
	-D NTHREADS=1 \
	./src/omp_homework.c -o ./release/omp_homework 
DFTW calculation with N = 20000 
DFTW computation in 1.751113 seconds
Xre[0] = 20000.000000 

STDERR:

nthreads=1, schedule=guided, size=20000
STDOUT:
icx -g -Wall -qopenmp -qopt-report=3 -march=native -O3 -ffast-math \
	-D SIZE=20000 \
	-D PARALLEL \
	-D OMP_SCHEDULE=guided \
	-D NTHREADS=1 \
	./src/omp_homework.c -o ./release/omp_homework 
DFTW calculation with N = 20000 
DFTW computation in 1.733544 s

OSError: [Errno 122] Disk quota exceeded

In [16]:
best_cpu_param_times

array([[3400., 3397., 3403.],
       [1703., 1729., 1708.],
       [1407.,  866.,  859.],
       [1092.,  582.,  571.],
       [ 898.,  440.,  437.],
       [ 705.,  380.,  376.],
       [ 670.,  338.,  356.],
       [ 564.,  312.,  316.],
       [ 498.,  288.,  291.],
       [ 523.,  267.,  266.],
       [ 394.,  252.,  251.]])

In [24]:
i_min, j_min = np.unravel_index(np.argmin(best_cpu_param_times), best_cpu_param_times.shape)
print(f"Best parameters: nthreads={NUM_THREADS_LIST[i_min]}, schedule={SCHEDULE_LIST[j_min]}")

ValueError: too many values to unpack (expected 2)